Hacer un modelo que tenga los siguientes pasos para precedir la probabilidad de sobrevivir en el titanic:

- Seleccion de columnas con mayor numero de varianza (hiperparametro el numero de columnas seleccionadas).
- Estandarizacion
- Knn


In [46]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
X,y = load_wine(as_frame=True, return_X_y=True)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=0, stratify=y)
print('Total de filas: ',X.shape[0],'Total de columnas: ',X.shape[1])
X

Total de filas:  178 Total de columnas:  13


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [47]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [48]:
X.std().sort_values().index[-2:]

Index(['magnesium', 'proline'], dtype='object')

In [49]:
class Most_variance_selector(BaseEstimator, TransformerMixin):
    
    def __init__(self,n_columns=0):
        self.n_columns = n_columns
    
    def fit(self,X,y=None):
        if self.n_columns > X.shape[1]:
            self.n_columns = 0
        self.columns_selected = X.std().sort_values().index[-self.n_columns:]
        return self
    
    def transform(self, X, y=None):
        return X[self.columns_selected]

In [50]:
pasos = [('varianza', Most_variance_selector()),('scaler', None), ('knn', KNeighborsClassifier())]
pipe = Pipeline(pasos)

In [51]:
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=0)
param_grid = {'knn__n_neighbors':range(2,4,6),'knn__weights':['uniform','distance'], 'scaler':[None,StandardScaler(),MinMaxScaler()], 'varianza__n_columns':[0,1,2,3,4,5]}

In [52]:
grid = GridSearchCV(pipe, param_grid, cv=folds)
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('varianza', Most_variance_selector()),
                                       ('scaler', None),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': range(2, 4, 6),
                         'knn__weights': ['uniform', 'distance'],
                         'scaler': [None, StandardScaler(), MinMaxScaler()],
                         'varianza__n_columns': [0, 1, 2, 3, 4, 5]})

In [53]:
print('Score: ',grid.best_score_)
grid.best_estimator_

Score:  0.9193333333333333


Pipeline(steps=[('varianza', Most_variance_selector()),
                ('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=2))])